# Проект для «Викишоп» с BERT

**Цель проекта**

Подготовить прототип модели машинного обучения для интернет-магазина «Викишоп», запускающего новый сервис по редактированию и дополнению описаний товаров пользователями, как в вики-сообществах. Модель должна  классифицировать комментарии на позитивные и негативные, чтобы отправлять негативные комментарии на модерацию. 

При определении оптимальной модели должен быть учтен важный для заказчика параметр:
качество предсказания, оцененное метрикой качества F1, должно быть не меньше 0.75.

**Данные для анализа**

На анализ передан набор данных с разметкой о токсичности правок.

**Шаги (план) проекта**

1.  Подготовка данных
2.  Обучение и проверка моделей на обучающей выборке
3.  Проверка модели на тестовой выборке и итоговый вывод

## Подготовка данных

Импортируем библиотеки, необходимые для работы с данными в текущем проекте.

In [1]:
# импорт библиотек
import numpy as np
import pandas as pd

# предобработка текста
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = stopwords.words('english')
import re 

# векторизация текста
import torch
import transformers as ppb

# борьба с дисбалансом
from sklearn.utils import shuffle

# обучение моделей
import lightgbm as ltb
from lightgbm import LGBMClassifier
from numpy.random import RandomState
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# оценка качества модели
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
import sklearn.metrics as metrics

# отключение предупреждений
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Maris\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Maris\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Установим RandomState равным 12345.

In [2]:
# фиксирование RandomState
state = np.random.RandomState(12345)

### Загрузка и изучение данных

Откроем датасет с данными и изучим его. C:\Users\Maris\Desktop\Обучение

In [3]:
# открытие файла с применением конструкции try/except для исключения проблем с отработкой кода при подготовке проекта локально
try:
    df = pd.read_csv('/datasets/toxic_comments.csv')
except FileNotFoundError:
    df = pd.read_csv('C:/Users/Maris/Desktop/Обучение/toxic_comments.csv')
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Изучим общую информацию о данных.

In [4]:
# получение общей информации о структуре датафрейма
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


1. В датасете содержится 159571 строк, 2 столбца.
2. Колонки названы в соответствии с правилами нотации Python (латиница, нижний регистр), переименование не требуется.
3. В колонке toxic требуется изменение типа данных int64 на int32 для экономии памяти.
4. Пропущенных значений нет, обработка пропусков не требуется.
5. Требуется проверка данных на наличие явных дубликатов.

Изменим тип данных в колонке toxic с int64 на int32 для экономии памяти.

In [5]:
# изменение типов данных
df[df.select_dtypes(np.int64).columns] = df.select_dtypes(np.int64).astype(np.int32)

Проверим результат.

In [6]:
# получение общей информации о структуре датафрейма
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int32 
dtypes: int32(1), object(1)
memory usage: 1.8+ MB


Изменение типов данных произведено корректно.

Проверим датафрейм на наличие явных дубликатов.

In [7]:
# проверка наличия явных дубликатов
df.duplicated().sum()

0

Дубликатов нет, удаление не требуется.

### Подготовка данных для обучения

**Предобработка текстов**

Прежде, чем векторизировать тексты, проведем их предобработку: приведем все тексты к нижнему регистру, удалим небуквенные символы и стоп-слова.

In [8]:
# перевод всех букв текстов корпуса в нижний регистр
df['text'] = df['text'].str.lower()

In [9]:
# подготовка функции для удаления символов из текстов (кроме букв)
def clear_text(text):
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    clear_text = " ".join(text.split())
    return clear_text

In [10]:
# очистка текстов от символов (кроме букв)
df["text"] = df["text"].apply(lambda x: clear_text(x))

In [11]:
# удаление стоп-слов из текстов корпуса
pat = r'\b(?:{})\b'.format('|'.join(stopwords))
df["text"] = df["text"].str.replace(pat, '')

Оценим результат.

In [12]:
# вывод первых 5 строк датафрейма
df.head()

,text,toxic
0,explanation edits made username hardcore m...,0
1,aww matches background colour seemingly s...,0
2,hey man really trying edit war guy c...,0
3,make real suggestions improvement wonde...,0
4,sir hero chance remember page,0


Предобразование текстов проведено успешно.

**Векторизация текстов**

Воспользуемся предобученной моделью DistilBERT для векторизации текстов. Обрежем выборку с учетом ограничений используемой модели векторизации (длина последовательности токенов не более 512).

Для оптимизации работы и во избежание проблем с нехваткой памяти сократим объем выборки до 2 000 случайных записей.

In [13]:
# выбор 2 000 случайных текстов из выборки
df_sample = df.sample(2000, random_state = 12345).reset_index(drop=True)

 Загрузим предобученную модель DistilBERT и токенизатор. 

In [14]:
# загрузка предобученной модели DistilBert с токенайзером
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Осуществим токенизацию текстов в корпусе.

In [15]:
# замена каждого токена его идентификатором из таблицы эмбеддингов
tokenized = df_sample['text'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True, max_length=512, truncation=True)))
tokenized.head()

0    [101, 6289, 2232, 3844, 6616, 2079, 19140, 160...
1    [101, 7514, 2518, 3146, 6236, 2924, 3190, 1065...
2    [101, 7514, 4931, 2071, 2560, 5254, 18626, 134...
3            [101, 2008, 2015, 2986, 15117, 9610, 102]
4    [101, 1040, 15922, 6488, 2442, 5400, 5638, 221...
Name: text, dtype: object

Проверим выполнение условия по максимальной длине последовательности токенов для модели DistilBERT.

In [16]:
# вывод длины максимально длинной последовательности токенов в корпусе
n = len(max(tokenized, key=len))
n

512

Приведем векторы к одной длине, дополнив их нулями.

In [17]:
# приведение векторов к одному размеру путем прибавления к более коротким векторам идентификатора 0
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

Сформируем маску для того, чтобы выделить действительно важные токены (укажем нулевые и не нулевые значения).

In [18]:
attention_mask = np.where(padded != 0, 1, 0)

Создадим входной вектор из матрицы токенов и передадим его модели DistilBERT. Сформируем эмбеддинги, для оптимизации памяти разбив корпус на батчи, и сформируем массив признаков из эмбеддингов всех текстов.

In [19]:
# получение эмбеддингов текстов корпуса
batch_size = 100
results = []
for i in range(padded.shape[0] // batch_size):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        mask = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])

        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=mask)

        results.append(batch_embeddings[0][:,0,:].numpy())

features = np.concatenate(results)

Оценим размер получившегося массива.

In [20]:
# вывод размеров массива
features.shape

(2000, 768)

**Подготовка тестовой, валидационной и обучающей выборок**

Выделим целевой признак из выборки.

In [21]:
# выделение целевого признака - тональности комментария
labels = df_sample['toxic']

Оценим размер получившегося датафрейма.

In [22]:
# вывод размеров датафрейма
labels.shape

(2000,)

Проверим сбалансированность классов в выборке с целевым признаком.

In [23]:
# расчет процентного соотношения классов
labels.value_counts(normalize=True)

0    0.904
1    0.096
Name: toxic, dtype: float64

Выборка несбалансирована: негативных комментариев в 10 раз больше. Это потребуется учесть при дальнейшем обучении, применив порог классификации.

Разделим выборку на обучающую, валидационную и тестовую.

In [24]:
# разделение выборки на обучающую и 20% для валидационной и тестовой выборок
features_train, features_tv, target_train, target_tv = train_test_split(
    features, labels, test_size=0.20, random_state=state)

In [25]:
# разделение отобранных 20% на валидационную и тестовую выборки
features_valid, features_test, target_valid, target_test = train_test_split(
    features_tv, target_tv, test_size=0.50, random_state=state)

Оценим размеры получившихся выборок.

In [26]:
# вывод размера выборки
features_train.shape

(1600, 768)

In [27]:
# вывод размера выборки
features_valid.shape

(200, 768)

In [28]:
# вывод размера выборки
features_test.shape

(200, 768)

In [29]:
# вывод размера выборки
target_train.shape

(1600,)

In [30]:
# вывод размера выборки
target_valid.shape

(200,)

In [31]:
# вывод размера выборки
target_test.shape

(200,)

Разделение на выборки прошло корректно.

<div style="border:solid blue 2px; padding: 20px">
    
**Вывод** 

В рамках данного этапа были проделаны следующие работы:
    
1. Открыт файл с данными и проведен их анализ (на вход поступила выборка из 159571 строки, 2 столбцов), по его итогам  изменен тип данных в колонке toxic для экономии памяти; пропусков и дублей в данных не обнаружено.
2. Проведена предобработка текстов в корпусе: все буквы приведены к нижнему регистру, из текстов удалены все прочие символы (кроме букв), а также стоп-слова.
3. Проведена векторизация текстов с учетом ограничений модели DistilBERT: последовательности токенов обрезаны до 512. С учетом ограничений по оперативной памяти компьютера корпус для дальнейшей работы сокращен до 2000 векторов.
4. Проведено разделение признаков и целевого признака на обучающую, валидационную и тестовую выборки (валидационная и тестовая выборки по 10%).
5. Выборка сильно несбалансирована по классам (90% - негативная тональность, 10% - позитивная тональность), что потребуется учесть при обучении модели.
      
<div>

## Обучение и проверка моделей на обучающей выборке

Обучим и протестируем на обучающей выборке модели **Логистическая регрессия, Случайный лес и LGBM.**

**Логистическая регрессия**

In [32]:
# создание модели
model_reg = LogisticRegression(random_state=state, solver='liblinear', class_weight='balanced')

In [33]:
# создание оценщика на основе f1_score
f1_scorer = make_scorer(f1_score)

In [34]:
# обучение модели и оценка F1-score на обучающей выборке с помощью кросс-валидации
result_train_reg = cross_val_score (model_reg, features_train, target_train, cv=5, scoring=f1_scorer).mean()
print("Логистическая регрессия_F1-score на обучающей выборке:", result_train_reg)

Логистическая регрессия_F1-score на обучающей выборке: 0.5844264821274946


In [35]:
# обучение модели и оценка F1-score на обучающей выборке с учетом порога классификации
model_reg.fit(features_train, target_train)
THRESHOLD = 0.75
predictions = np.where(model_reg.predict_proba(features_train)[:,1] > THRESHOLD, 1, 0)
print("Логистическая регрессия_F1-score на обучающей выборке с учетом порога:", f1_score(target_train, predictions))

Логистическая регрессия_F1-score на обучающей выборке с учетом порога: 0.8963210702341137


Проверим работу модели также и на валидационной выборке с учетом порога классификации.

In [36]:
# оценка F1-score на валидационной выборке с учетом порога классификации
THRESHOLD = 0.75
predictions = np.where(model_reg.predict_proba(features_valid)[:,1] > THRESHOLD, 1, 0)
print("Логистическая регрессия_F1-score на валидационной выборке с учетом порога:", f1_score(target_valid, predictions))

Логистическая регрессия_F1-score на валидационной выборке с учетом порога: 0.7999999999999999


**Случайный лес**

In [37]:
# подбор параметров модели с помощью GridSearchCV
clf = RandomForestClassifier(random_state=state)
parameters = {'n_estimators': range (10, 51, 10),
              'max_depth': range (1,13,2)}
grid = GridSearchCV(clf, parameters, cv=5, scoring=f1_score, n_jobs=-1)
grid.fit(features_train, target_train)
grid.best_params_

{'max_depth': 1, 'n_estimators': 10}

In [38]:
# сознание модели
model_forest = RandomForestClassifier(random_state=state, max_depth=1, n_estimators=10)

In [39]:
# обучение модели и оценка F1-score на обучающей выборке с помощью кросс-валидации
result_train_forest = cross_val_score (model_forest, features_train, target_train, cv=5, scoring=f1_scorer).mean()
print("Случайный лес_F1-score на обучающей выборке:", result_train_forest)

Случайный лес_F1-score на обучающей выборке: 0.0


**LGBM**

In [40]:
# подбор параметров с помощью GridSearchCV
clf = ltb.LGBMClassifier(random_state=state)
parameters = {
    'max_depth': [5, 9],
    'n_estimators': [50, 100],
}
grid = GridSearchCV(clf, parameters, cv=5, scoring=f1_score, n_jobs=-1)
grid.fit(features_train, target_train)
grid.best_params_

{'max_depth': 5, 'n_estimators': 50}

In [41]:
# сознание модели
model_LGBM = ltb.LGBMClassifier(random_state=state, max_depth=5, n_estimators=50)

In [42]:
# обучение модели и оценка F1-score на обучающей выборке с помощью кросс-валидации
result_train_LGBM = cross_val_score (model_LGBM, features_train, target_train, cv=5, scoring=f1_scorer).mean()
print("LGBM_F1-score на обучающей выборке:", result_train_LGBM)

LGBM_F1-score на обучающей выборке: 0.4947937181235167


<div style="border:solid blue 2px; padding: 20px">
    
**Вывод** 

В рамках данного этапа были проделаны следующие работы:
1. На данных обучающей выборки проведено обучение моделей Логистической регрессии, Случайного леса и LGBM. Результаты приведены в таблице:
    
|Модель                                   |Качество предсказания (F1 score)|
|:---------------------------------------:|-------------------------------:|
|Логистическая регрессия                  |0.58                            |
|Логистическая регрессия с учетом порога  |0.89                            |
|Случайный лес max_depth=1, n_est=10      |0.00                            |
|LGBM max_depth=5, n_est=50               |0.49                            |  

2. Также проведено обучение и проверка модели Логистической регрессии на валидационной выборке. F1-score составил 0.79. 
    
Таким образом, модель Логистической регрессии с учетом порога классификации показала наилучшие результаты, которые мы проверим на следующем шаге на данных тестовой выборки.      
   
<div>

## Проверка модели на тестовой выборке и итоговый вывод

По итогам обучения и проверки на обучающей и валидационной выборках наилучший результат показала модель **Логистической регрессии** с учетом порога классификации. Оценим ее результаты на данных тестовой выборки.

In [43]:
THRESHOLD = 0.75
predictions = np.where(model_reg.predict_proba(features_test)[:,1] > THRESHOLD, 1, 0)
print("Логистическая регрессия_F1-score на тестовой выборке с учетом порога:", f1_score(target_test, predictions))

Логистическая регрессия_F1-score на тестовой выборке с учетом порога: 0.8372093023255814


<div style="border:solid blue 2px; padding: 20px">
    
**Вывод** 

В рамках данного этапа проведена проверка результативности модели Логистической регрессии с учетом порога на тестовых данных. Итоговое значение метрики F1 на тестовых данных составило 0.83, что соответствует условию задачи (f1_score не менее 0.75).
    
Таким образом, данная модель может быть рекомендована для дальнейшего использования сервисом.    
   
<div>